In [ ]:
# Load libraries
import os
import numpy as np
import pickle as pkl
import nibabel as nib
from tqdm.notebook import tqdm
import random

# Load plotting
import matplotlib.pyplot as plt
import seaborn as sns

# For parallel computations
from joblib import Parallel, delayed, parallel_backend
import dask

# Sklearn utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# Load Tensorflow
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.backend import int_shape
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, Conv2DTranspose,
    MaxPooling2D, Dropout, Input, concatenate, Cropping2D
)

from tensorflow.keras import backend as K
from keras.optimizers import Adam, SGD

from livelossplot import PlotLossesKeras

In [ ]:
# Check for GPU support
if tf.test.gpu_device_name(): 
    print('Default GPU Device: \
    {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

In [ ]:
# Load custom Unet architecture
from custom_unet import *

In [ ]:
# Load custom metric functions
from metrics import *

In [ ]:
# Load custom helper functions
from utility import *

### Download data

In [ ]:
os.makedirs("./data", exist_ok=True)
data_path = "./data/ml4h_proj1_colon_cancer_ct"

In [ ]:
# Download and unpack data stored in given Google Drive Directory
import tarfile
from google_drive_downloader import GoogleDriveDownloader as gdd

print("Downloading from Google Drive...")
gdd.download_file_from_google_drive(
    file_id='', # data not publicly available
    dest_path=data_path + ".tar.gz",
    showsize=True
)

print("Unpacking...", end='')
tar = tarfile.open(data_path + ".tar.gz", "r:gz")
tar.extractall(path="./data")
tar.close()

print("Done")

### Import data

In [ ]:
# Load training data (images and labels) using helper function
imgs, lbls = read_training_data_parallel(data_path, njobs=32, frac=None, load_scaled=True)
print(len(imgs), len(lbls), imgs[0].shape)

Separate some patients for validation

In [ ]:
imgs_train, imgs_valid, lbls_train, lbls_valid = train_test_split(
    imgs,
    lbls,
    test_size=0.2,
    random_state=42
)
print("Number of training/validation patients", len(imgs_train), len(imgs_valid))

Load test images

In [ ]:
test_data = read_testing_data_parallel(data_path, njobs=8, frac=None, load_scaled=True, get_names=True)
print(len(test_data), test_data[0][1].shape)

### Analyze depth distribution

In [ ]:
# Different depth values:
img_depths = set()
for img in imgs_train:
    img_depths.add(img.shape[2])

print("Image depths: ", min(img_depths), "to", max(img_depths))

### Analyze imbalance

In [ ]:
imgs_class = [1 if np.sum(lbl) > 0 else 0 for lbl in lbls_train]

In [ ]:
print("Patients with cancerous tissue: {:.2f}%".format(sum(imgs_class)/len(imgs_class) * 100))

In [ ]:
cancerous_layers = []
for lbl in lbls:
    cancerous_layers.append(len(get_cancerous_layers(lbl)))

In [ ]:
sns.histplot(cancerous_layers)
print("Mean number of cancerous layers:", np.mean(cancerous_layers))

## Preprocessing

Split 3D images into separate layers considering two neighboring layers on each side for the input image and
one neighbor on each side for the output segmentation mask. The model will thus learn a mapping from 5 channel 3D slices
to 3 channel 3D slices 

In [ ]:
NEIGHBORS = 2
OUTPUT_NEIGHBORS = 1

In [ ]:
train_imgs_sep = convert_depth_to_imgs_keras(imgs_train, neighbors=NEIGHBORS)
valid_imgs_sep = convert_depth_to_imgs_keras(imgs_valid, neighbors=NEIGHBORS)

In [ ]:
train_lbls_sep = convert_depth_to_imgs_keras(lbls_train, neighbors=OUTPUT_NEIGHBORS)
valid_lbls_sep = convert_depth_to_imgs_keras(lbls_valid, neighbors=OUTPUT_NEIGHBORS)

#### Analyze balance on converted samples

In [ ]:
# Check how many samples actually contain cancerous tissue (balance check)
count = sum([1 for l in train_lbls_sep if np.sum(l) > 0.0])
print("Train ratio of images with canc. tissue {:.2f}% of {} images".format(count/len(train_lbls_sep) * 100, len(train_lbls_sep)))

count = sum([1 for l in valid_lbls_sep if np.sum(l) > 0.0])
print("Valid ratio of images with canc. tissue {:.2f}% of {} images".format(count/len(valid_lbls_sep) * 100, len(valid_lbls_sep)))

#### Downsample
Downsample the training and validation dataset to contain all layers with a cancerous
segmentation result and a given ratio of images without a cancerous segmentation mask

In [ ]:
# Two different configurations have been used to train the two models of the ensemble

# Configuration one (Model A and B in the report)
# frac = 0.8
# strategy = None

# Configuration two (Model C in the report)
frac = None
strategy = {0: 500, 1: 950}

imgs_down_train, lbls_down_train = downsample(train_imgs_sep, train_lbls_sep, frac=frac, strategy=strategy)

## Train a custom U-Net architecture
Copied and adjusted from [Github: karolzak/keras-unet](https://github.com/karolzak/keras-unet)

We refer to the naming adopted in our report. Three models are essential to the final results, all use the same network architecture with only slight modifications to the hyperparameters of the loss and the downsampling of the data, as well as the weight initialization
- **Model A**
    - Uses *random* weight initialization
    - Trained with downsampled dataset of around 45% layers containing cancerous tissue
    - Configures the loss with $\beta=1.0$
    - 100 epochs
    - Achieves final validation 3D IoU of around ~0.15
    - Named Model 5, when loading and saving
- **Model B**
    - Uses *pretrained* weight initialization
    - Trained with downsampled dataset of around 45% layers containing cancerous tissue
    - Configures the loss with $\beta=1.0$
    - 100 epochs
    - Achieves final validation 3D IoU of around ~0.17
    - Named Model 6, when loading and saving
- **Model C**
    - Uses *pretrained* weight initialization
    - Trained with downsampled dataset of around 65% layers containing cancerous tissue
    - Configures the loss with $\beta=2.0$
    - 80 epochs
    - Achieves final validation 3D IoU of around ~0.16
    - Named Model 7, when loading and saving
- **Ensemble, Model D**
    - Used for final predictions, averages predictions of Model B and C before thresholding

### Loss
Use **Focal Tversky Loss** implementation from here: https://www.kaggle.com/bigironsphere/loss-function-library-keras-pytorch
A combination of Focal Loss and Tversky Loss, which has been found to work well for a medical setting where the goal is 
to segment structures which are small and delicate compared to the overall image size

#### Focal Tversky Loss

In [ ]:
#Keras
ALPHA = 0.5 # Penalize False Positives
BETA = 2.0  # Penalize False Negatives; two configurations have been used Beta=1.0 (Model A and B) and Beta=2.0 (Model C) for the respective models of the ensemble
GAMMA = 4.0 # Focus on wrong predictions

def FocalTverskyLoss(targets, inputs, alpha=ALPHA, beta=BETA, gamma=GAMMA, smooth=1e-6):
    
        #flatten label and prediction tensors
        inputs = K.flatten(inputs)
        targets = K.flatten(targets)
        
        #True Positives, False Positives & False Negatives
        TP = K.sum((inputs * targets))
        FP = K.sum(((1-targets) * inputs))
        FN = K.sum((targets * (1-inputs)))
               
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        FocalTversky = K.pow((1 - Tversky), gamma)
        
        return FocalTversky

### Load validation data

In [ ]:
# Validate on full patients (i.e. include all layers of each validation patient)
# Load into contiguous memory
valid_x = np.array(valid_imgs_sep, dtype=np.float32)
valid_y = np.array(valid_lbls_sep, dtype=np.float32)

# Save memory
del valid_imgs_sep
del valid_lbls_sep

print("Validation data:", len(valid_x))

### Data augmentation
Apply transformations to the training data:

- Random left/right flipping
- Random up/down flipping
- Random rotations
- Add Gaussian Noise (Std=0.05)

In [ ]:
def apply_transformations(img, lbl):
    
    # Flip left-right randomly
    choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)
    img = tf.cond(choice < 0.5, lambda: img, lambda: tf.image.flip_left_right(img))
    lbl = tf.cond(choice < 0.5, lambda: lbl, lambda: tf.image.flip_left_right(lbl))
    
    # Flip up-down randomly
    choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)
    img = tf.cond(choice < 0.5, lambda: img, lambda: tf.image.flip_up_down(img))
    lbl = tf.cond(choice < 0.5, lambda: lbl, lambda: tf.image.flip_up_down(lbl))
    
    # Rotate by random angle
    angle = tf.random.uniform(shape=[], minval=0, maxval=360, dtype=tf.int32)
    angle = tf.dtypes.cast(angle, tf.float32)
    
    img = tfa.image.rotate(img, angle)
    lbl = tfa.image.rotate(lbl, angle)
    
    # Add noise to image
    noise = tf.random.normal(shape=tf.shape(img), mean=1.0, stddev=0.05, dtype=tf.float32)
    noise_img = tf.dtypes.cast(img, tf.float32) * noise
    
    return (noise_img, lbl)

#### Apply transformations and prepare training data
We double the training data and perturb each sample differently, then load into contiguous memory

In [ ]:
train_x = []
train_y = []
for x, y in tqdm(zip(imgs_down_train, lbls_down_train), total=len(imgs_down_train)):
    x_out, y_out = apply_transformations(x, y)
    train_x.append(np.array(x_out))
    train_y.append(np.array(y_out))
    
for x, y in tqdm(zip(imgs_down_train, lbls_down_train), total=len(imgs_down_train)):
    x_out, y_out = apply_transformations(x, y)
    train_x.append(np.array(x_out))
    train_y.append(np.array(y_out))
    
train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

# Save memory
del imgs_down_train
del lbls_down_train

print("Training data:", len(train_x))

### Model and Training

In [ ]:
# Custom Network, Loss and Metrics used (for model loading)
custom_objects = {
    'custom_unet' : custom_unet,
    'FocalTverskyLoss': FocalTverskyLoss,
    'iou_thresholded' : iou_thresholded,
    'iou' : iou
}

#### Load pretrained weights
Weights have been pretrained on a dataset of CT scans with trachea segmentations

- Model B and C both use the below pretrained network for weight initialization
- Model A has been trained using random initialization of the network use the subsequent cell to randomly initialize the network

In [ ]:
strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1", "GPU:2", "GPU:3"])
with strategy.scope():
    unet = keras.models.load_model('./model/pretrained_transfer_trachea_1', custom_objects=custom_objects)

#### Network loading
**Careful** please do not execute the following cell, if you wish to train using the pretrained weights. It is only here for reference of the model layout. Executing it, will randomly initialize the model weights.

In [ ]:
# Load network
# strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1", "GPU:2", "GPU:3"])
# with strategy.scope():
    
#     unet = custom_unet(
#         train_x[0].shape,
#         num_classes=OUTPUT_NEIGHBORS * 2 + 1, # Number of output channels
#         filters=64, # Number of filters in the first Conv. Block, doubled with each block
#         use_batch_norm=True, # use batch normalization
#         dropout=0.2,  # Use this amount of dropout
#         dropout_change_per_layer=0.0, # Do not increase dropout on subsequent blocks
#         dropout_type='spatial', # Use spatial dropout i.e. drop entire convolutional filters
#         num_layers=4, # Number of Conv. Blocks, 4 is default for vanilla Unet
#         upsample_mode='deconv', # Use transposed convolutions on upsampling part of network
#         use_dropout_on_upsampling=False # No dropout in upsampling section
#     )
    
#     unet.compile(
#         optimizer=Adam(),
#         loss=FocalTverskyLoss,
#         metrics=[iou, iou_thresholded, tf.keras.metrics.AUC()]
#     )
    
# unet.summary()

In [ ]:
# Store checkpoints for best validation loss model
save_best_cb = tf.keras.callbacks.ModelCheckpoint(
    './model/best_checkpoint', monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='min', save_freq='epoch'
)

#### Run Training
- To train models A and B: 100 epochs have been run
- To train model C: 80 epochs have been run

In [ ]:
EPOCHS = 80
BATCH_SIZE = 24 # 128

history = unet.fit(
    train_x, train_y,
    epochs=EPOCHS,
    batch_size = BATCH_SIZE,
    validation_data=(valid_x, valid_y),
    callbacks=[PlotLossesKeras(), save_best_cb]
)

### Store model

In [ ]:
MODEL_NUMBER = 8

In [ ]:
os.makedirs("./model", exist_ok=True)
unet.save(f"./model/promising_model_{MODEL_NUMBER}_transfer", overwrite=False)

### Load Fully Trained Models
For ensembling Model B and C (corresponds to Model D) during prediction

In [ ]:
# Load Model C (was model 7 in the development iteration process, thus the below naming)
# strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1"])
# with strategy.scope():
#     unet7 = keras.models.load_model('./model/promising_model_7_transfer_80', custom_objects=custom_objects)

In [ ]:
# Load Model B (was model 6 in the development iteration process, thus below naming)
# strategy = tf.distribute.MirroredStrategy(["GPU:2", "GPU:3"])
# with strategy.scope():
#     unet6 = keras.models.load_model('./model/promising_model_6_transfer', custom_objects=custom_objects)

## Inspect and compute scores on validation

In [ ]:
def make_prediction(samples, threshold=0.5):
    
    out = unet.predict(samples)
    out = threshold_binarize(out, threshold=threshold)
    
    return out

In [ ]:
def make_subplots(plot_imgs, i, depth=0):
    f, axarr = plt.subplots(1,8, figsize=(15,15))

    axarr[0].imshow(plot_imgs[i][:,:,depth])
    axarr[1].imshow(plot_imgs[i+1][:,:,depth])
    axarr[2].imshow(plot_imgs[i+2][:,:,depth])
    axarr[3].imshow(plot_imgs[i+3][:,:,depth])
    axarr[4].imshow(plot_imgs[i+4][:,:,depth])
    axarr[5].imshow(plot_imgs[i+5][:,:,depth])
    axarr[6].imshow(plot_imgs[i+6][:,:,depth])
    axarr[7].imshow(plot_imgs[i+7][:,:,depth])

In [ ]:
def make_plots_depth(img, depth=3, offset=0):
    f, axarr = plt.subplots(1, depth, figsize=(15,15))
    
    for i in range(depth):
        axarr[i].imshow(img[:,:,offset+i])

#### Analyze threshold response

In [ ]:
# predict_y = unet.predict(valid_x)
# predict_tresh = threshold_binarize(predict_y, threshold=0.5)

In [ ]:
# def threshold_response_scores(valid_data, predict_data):
#     thresholds = np.linspace(0.1, 1.0, 20)
#     scores = []
#     for t in tqdm(thresholds):
#     #     score = iou_thresholded(valid_y, predict_y, threshold=t)
#         score_f = lambda x: iou_thresholded(x[0], x[1], threshold=t)
#         score = map(score_f, zip(valid_data, predict_data))
#         score = np.mean(list(score))
#         scores.append(float(score))
        
#     return scores

In [ ]:
# scores = threshold_response_scores(valid_y, predict_y)
# thresholds = np.linspace(0.1, 1.0, 20)
# ax = sns.lineplot(x=tresholds, y=scores)
# ax = ax.set(xlabel='Treshold', ylabel='IoU')

#### ROC

In [ ]:
# from sklearn.metrics import roc_curve, auc

In [ ]:
# fpr, tpr, tresh_roc = roc_curve(valid_y.ravel(), predict_y.ravel())
# roc_auc = auc(fpr, tpr)
# print("AuROC: {

In [ ]:
# plot(x=fpr[0::1000], y=tpr[0::1000])
# ax = ax.set(xla

#### Visual inspection

In [ ]:
# make_subplots(valid_y, INDEX)

In [ ]:
# make_subplots(predict_y, INDEX)
# make_subplots(predict_y_checkpoint, INDEX)

In [ ]:
# make_subplots(predict_tresh, INDEX)
# make_subplots(predict_tresh_checkpoint, INDEX)

### Compute 3d IoU over validation

In [ ]:
# Clamping function
def clamp(n, minn, maxn):
    return max(min(maxn, n), minn)

In [ ]:
# Predict for a single patient using a given network
# For each layer we overlap and average the predictions
# for the same layer due to the 3 channel output of the model
def predict_patient_averaged(img, net, threshold=0.5):
    
    sep = convert_depth_to_imgs_keras([img], neighbors=NEIGHBORS, print_shape=False)
    pred = net.predict(np.array(sep, dtype=np.float32))
    
    # Compute a single averaged layer considering the outputs
    # of the neighboring layer predictions and channels
    def compute_averaged_layer(predictions, i_layer):
        
        tmp = []
        for j in range(-OUTPUT_NEIGHBORS, OUTPUT_NEIGHBORS+1):
            
            layer = i_layer + j
            channel = j * -1 + OUTPUT_NEIGHBORS
            if i_layer == 0 and layer < 0:
                channel = OUTPUT_NEIGHBORS
            if i_layer == predictions.shape[0]-1 and layer >= predictions.shape[0]:
                channel = OUTPUT_NEIGHBORS
            
            layer = clamp(layer, 0, predictions.shape[0]-1)
            tmp.append(predictions[layer][:,:,channel])
            
            
        tmp = np.array(tmp, dtype=np.float32)
        return np.mean(tmp, axis=0)
        
    
    out = []
    # Compute the averaged layer for each layer for given patient
    for i in range(pred.shape[0]):       
        layer_avg = dask.delayed(compute_averaged_layer)(pred, i)
        out.append(layer_avg)
    
    out = dask.compute(*out, num_workers=24)
    
    output = np.moveaxis(np.array(out, dtype=np.float32), 0, -1)  
        
    return output
            

In [ ]:
# Compute the 3D IoU given a patient and its labels
def compute_3d_IoU(img, val, threshold=0.5):
    
    pred = predict_patient_averaged(img, unet, threshold=threshold)
    
    smooth=1.
    y_true_f = K.flatten(np.array(val, dtype=np.float32))
    y_pred_f = K.flatten(pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)
    
    return float(score)

In [ ]:
# Compute validation 3D IoU score using the ensemble of models
def compute_validation_3dscore(t):
    scores = []
    for img, val, i in tqdm(zip(imgs_valid, lbls_valid, range(len(imgs_valid))), total=len(imgs_valid)):

        # Predict with both models B and C for ensembling into Model D
        pred6 = predict_patient_averaged(img, unet6, threshold=t) # Model B
        pred7 = predict_patient_averaged(img, unet7, threshold=t) # Model C
        
        # Average predictions and threshold
        output = np.mean([pred6, pred7], axis=0)  
        pred = threshold_binarize(output, threshold=t)

        smooth=1.
        y_true_f = K.flatten(np.array(val, dtype=np.float32))
        y_pred_f = K.flatten(pred)
        intersection = K.sum(y_true_f * y_pred_f)
        score = (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

        print(f"Patient {i}: {score}")
        scores.append(float(score))
    
    return scores

In [ ]:
scores = compute_validation_3dscore(0.5)
print("Validation Mean 3D IoU:", np.mean(scores))

#### Analyze diff. thresholds

In [ ]:
# thresholds = [0.4, 0.5, 0.6, 0.7]
# scores_3d = []
# for t in tqdm(thresholds):
#     scores_3d.append(np.mean(compute_validation_3dscore(t)))

In [ ]:
# best_t = max(zip(thresholds, scores_3d), key=lambda x: x[1])
# print("Best treshold:", best_t[0], " with score", best_t[1])

In [ ]:
# ax = sns.lineplot(x=thresholds, y=scores_3d)
# ax = ax.set(xlabel='Treshold', ylabel=' 3D IoU')

# print(scores_3d)

### Predict on TEST and store
Use the ensemble of Model B and Model C (i.e. model 6 and model 7 from development process) to predict and store the results (ensemble refered to as Model D in the report)

In [ ]:
for patient_name, img in tqdm(test_data, total=len(test_data)):
    
    # Extract patient number
    patient_id = patient_name[0:9]
    
    # Predict with ensemble of Models B and C (i.e. use Model D)
    pred6 = predict_patient_averaged(img, unet6, threshold=0.5) # Model B
    pred7 = predict_patient_averaged(img, unet7, threshold=0.5) # Model C
    output = np.mean([pred6, pred7], axis=0)
    pred = np.array(threshold_binarize(output, threshold=0.5), dtype=np.float32)
    
    # Create folders
    folder = f"./predictions/model_{MODEL_NUMBER}_raw/prediction_test/patient_{patient_id}"
    os.makedirs(f"{folder}", exist_ok=True)
    os.makedirs(f"{folder}/raw", exist_ok=True)
    os.makedirs(f"{folder}/thresh", exist_ok=True)
    
    # Store pickle files of full 3D arrays
    thresh_file = open(f"{folder}/thresh_pickle_predict_patient_{patient_id}.pickle", "wb")
    raw_file = open(f"{folder}/raw_pickle_predict_patient_{patient_id}.pickle", "wb")
    pkl.dump(pred, thresh_file)
    pkl.dump(output, raw_file)
    thresh_file.close()
    raw_file.close()
    
    # Store individual layers as CSVs
    for j in range(pred.shape[2]):
        np.savetxt(f"{folder}/raw/raw_predict_patient_{patient_id}_layer_{j}.csv.gz", output[:,:,j], fmt="%1.6f")
        np.savetxt(f"{folder}/thresh/thresh_predict_patient_{patient_id}_layer_{j}.csv.gz", pred[:,:,j], fmt="%d")